In [2]:
import cv2
import imutils
from skimage.filters import threshold_local
from pyimagesearch.transform import four_point_transform
import numpy as np

Read and show the input image.

In [ ]:
img_path = 'b.jpg'
big_img = cv2.imread(img_path)
cv2.imshow('org img',big_img)
cv2.waitKey(0)

In [ ]:
ratio = big_img.shape[0] / 500.0 # ratio used later
org = big_img.copy() #keep a  copy of original image
# Resizing the image and making the height to 500 keeping the aspect ratio constant.
img = imutils.resize(big_img, height = 500)
cv2.imshow('resizing',img)
cv2.waitKey(0)

Here we are converting our BGR image to a grayscale image.
Here we are using Gaussian Blur to remove the Gaussian Noise from the image.
Also, we are simply finding the edges in the image using Canny Edge Detection.

In [4]:
gray_img = cv2.cvtColor(img.copy(),cv2.COLOR_BGR2GRAY)
blur_img = cv2.GaussianBlur(gray_img,(5,5),0)
edged_img = cv2.Canny(blur_img,75,200)


-1

In [ ]:
cv2.imshow('edged',edged_img)
cv2.waitKey(0)

Finding all the contours in the image and
 Sorting the contours in descending order based on their contour area and just taking the first 5.

In [ ]:
cnts,_ = cv2.findContours(edged_img.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
cnts = sorted(cnts,key=cv2.contourArea,reverse=True)[:5]

Traversing in contours and finding the contour with 4 sides using cv2.approxPolyDP().

In [3]:


for c in cnts:
    peri = cv2.arcLength(c,True)
    approx = cv2.approxPolyDP(c,0.02*peri,True)
    if len(approx)==4:
        doc = approx
        break
        

We are simply drawing points/circles around the corners of the document. These are the points that we got above in the contour detection step using the CHAIN_APPROX_SIMPLE method.

In [ ]:
p=[]
for d in doc:
    tuple_point = tuple(d[0])
    cv2.circle(img,tuple_point,3,(0,0,255),4)
    p.append(tuple_point)
cv2.imshow('Corner points detected',img)
cv2.waitKey(0)

 We are applying the four-point transformation to the image. It means that we will only extract the document from the image. 
 Or we can also say that we just want to extract the rectangle formed by the four points.

In [8]:
warped = four_point_transform(org, doc.reshape(4, 2) * ratio)
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
cv2.imshow("Warped", imutils.resize(warped, height = 650))
cv2.waitKey(0)

Threshold the image to give it a black and white feel.

In [ ]:
T = threshold_local(warped, 11, offset = 10, method = "gaussian")
warped = (warped > T).astype("uint8") * 255
cv2.imshow("Scanned", imutils.resize(warped, height = 650))
cv2.waitKey(0)

# destroy all widows
cv2.destroyAllWindows()